# Introducción

En esta práctica vamos a entrenar varios clasificadores sobre un mismo dataset, usando todos los algoritmos de clasificación que hemos visto hasta ahora e implementándolos usando las librerías adecuadas.

# Dataset

El dataset que usaremos es "Breast Cancer Wisconsin (Diagnostic)", disponible en [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic)
También está disponible directamente en `sklearn.datasets.load_breast_cancer()`.

**Descripción del dataset**


| Aspecto                 | Detalle                                                   |
| ----------------------- | --------------------------------------------------------- |
| **Tipo de problema**    | Clasificación binaria                                     |
| **Objetivo**            | Diagnosticar si un tumor es *maligno* (1) o *benigno* (0) |
| **Número de muestras**  | 569                                                       |
| **Número de variables** | 30 características numéricas                              |
| **Balance de clases**   | Moderadamente balanceado (~37% malignos, ~63% benignos)   |

# Métrica

Elige la métrica adecuada para evaluar tus modelos. Justifica tu respuesta.

El dataset no está perfectamente balanceada, por lo que accuracy puede inducir sesgos (un modelo que siempre predice “benigno” tendría ~63% de acierto).

El AUC-ROC mide la capacidad discriminativa del modelo (qué tan bien separa positivos y negativos) independientemente del umbral, y es más informativa cuando las clases son desiguales.

Es además aplicable y comparable entre modelos probabilísticos (Naïve Bayes, Regresión Logística, Redes Bayesianas) y no probabilísticos (CART).

# 1. Cargar el dataset e importar librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix

In [ ]:
# Cargar dataset
data = datasets.load_breast_cancer()

# Separar X e y
#X = 
#y =        
feature_names = data.feature_names
target_names = data.target_names

Información detallada sobre el dataset:

In [ ]:
print(data.DESCR)

Veamos cómo son las clases:

In [ ]:
print(data.target_names)

print(np.unique(data.target))

Por lo tanto, este dataset no sigue la convención usual y tenemos que:

0 -> 'malignant'

1 -> 'benign'

In [ ]:
# Cargar como dataframe, con y en la columna llamada 'target'
#df = 

#print("Clase (counts):")
#print(df['target'].value_counts())

In [ ]:
df.head()

# 2. Análisis exploratorio

In [ ]:
# Histograma por clase

In [ ]:
# Histogramas de algunas variables por clase


In [ ]:
# Todos los pairplots posibles, segmentados por clase.
# Usa Seaborn pairplot


In [ ]:
# Estadísticas descriptivas
print(df.describe().T[['mean', 'std', 'min', 'max']])

In [ ]:
# Matriz de correlaciones entre las variables
# Para calcular las correlaciones necesitas la función `corr``

In [ ]:
# Top correlaciones con el target

# 3. Separación train/test

In [ ]:
# Config reproducible
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:
# Separa train y test ysando la función adecuada de scikit-learn.
# Usa el parámetro random_state=RANDOM_STATE para controlar la aleatoriedad
# X_train, X_test, y_train, y_test = 

La regresión logística se entrena mediante descenso del gradiente, y la escala de las variables afecta la velocidad y estabilidad numérica del entrenamiento. Si las variables tienen órdenes de magnitud distintos, los gradientes también lo tendrán, y el proceso puede converger mal o lentamente.

Por este motivo, estandariza los datos usando `StandardScaler` —es decir, restar la media y dividir por la desviación típica.

In [ ]:
#scaler = 
#X_train_std = 
#X_test_std = 

# 4. Entrenar modelos clásicos


## Regresión logística

In [ ]:
# Usa LogisticRegression para ajustar una regresión logística


In [ ]:
# Realiza predicciones sobre test
# y_test_hat = 

Podemos ver no sólo la clase predicha, sino la probabilidad que asigna a cada clase:

In [ ]:
# Usa `predict_proba``
#y_test_proba = 

In [ ]:
# Cálculo de AUC
proba_log = log_clf.predict_proba(X_test_std)[:, 1]
auc_log = roc_auc_score(y_test, proba_log)
print(f"Logistic Regression AUC: {auc_log}")

## Naive Bayes (GaussianNB)

In [ ]:
# Naive Bayes usando GaussianNB
#nb_clf = 
#proba_nb = nb_clf.predict_proba(X_test_std)[:,1]
#auc_nb = roc_auc_score(y_test, proba_nb)
print(f"GaussianNB AUC: {auc_nb}")

## Árbol de decisión

In [ ]:
# Árbol de decisión (CART) usando DecisionTreeClassifier
#tree_clf = 

proba_tree = tree_clf.predict_proba(X_test_std)[:,1]
auc_tree = roc_auc_score(y_test, proba_tree)

print(f"CART AUC: {auc_nb}")

# 5. Red Bayesiana 

Implementa la red bayesiana con `pgmpy`.

In [ ]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_selection import mutual_info_classif

Seleccionamos un subconjunto de features (top K) para reducir la complejidad

In [ ]:
K = 6

mi = mutual_info_classif(X_train, y_train, random_state=42)
top_idx = np.argsort(mi)[-K:][::-1]
top_features = feature_names[top_idx]
print("Usando estas variables para la red bayesiana:", list(top_features))

Discretizamos estas features (KBins) usando cuantiles para mantener interpretación

In [ ]:
# Discretizar variables usando KBinsDiscretizer
kbd = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
X_train_disc = kbd.fit_transform(X_train[:, top_idx])
X_test_disc = kbd.transform(X_test[:, top_idx])

# Construimos DataFrame discreto para pgmpy
cols = [f"X{i}" for i in range(K)]
df_train_bn = pd.DataFrame(X_train_disc, columns=cols)
df_train_bn["target"] = y_train.astype(int)

In [ ]:
# Estructura de Naïve Bayes: target -> cada feature
edges = [("target", f"X{i}") for i in range(K)]
bn_model = DiscreteBayesianNetwork(edges)

# Ajustamos los parámetros por máxima verosimilitud
bn_model.fit(df_train_bn, estimator=MaximumLikelihoodEstimator)

# Inferencia
infer = VariableElimination(bn_model)

# Predicción de probabilidades para test
def bn_predict_proba_row(row_disc):
    evidence = {f"X{i}": int(row_disc[i]) for i in range(K)}
    q = infer.query(variables=["target"], evidence=evidence, show_progress=False)
    return float(q.values[1])  # probabilidad de clase 1 (benign)

proba_bn = np.array([bn_predict_proba_row(row) for row in X_test_disc])
auc_bn = roc_auc_score(y_test, proba_bn)
print(f"AUC - Red bayesiana (Naïve Bayes manual): {auc_bn:.4f}")


# 6. Comparación: ROC curves

In [ ]:
fpr_log, tpr_log, _ = roc_curve(y_test, proba_log)
fpr_nb, tpr_nb, _ = roc_curve(y_test, proba_nb)
fpr_tree, tpr_tree, _ = roc_curve(y_test, proba_tree)
fpr_bn, tpr_bn, _ = roc_curve(y_test, proba_bn)

plt.figure(figsize=(8,6))
plt.plot(fpr_log, tpr_log, label=f'Logistic (AUC={auc_log:.3f})')
plt.plot(fpr_nb, tpr_nb, label=f'GaussianNB (AUC={auc_nb:.3f})')
plt.plot(fpr_tree, tpr_tree, label=f'DecisionTree (AUC={auc_tree:.3f})')
plt.plot(fpr_bn, tpr_bn, label=f'BayesianNet (AUC={auc_bn:.3f})')
plt.plot([0,1], [0,1], 'k--', label='Chance')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves Comparison')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
# Resultado resumen en DataFrame

results = pd.DataFrame({
    'model': ['LogisticRegression','GaussianNB','DecisionTree','BayesianNetwork'],
    'auc': [auc_log, auc_nb, auc_tree, auc_bn]
})
print(results.sort_values('auc', ascending=False))